In [ ]:
import numpy as np
import imutils
import cv2
from matplotlib import pyplot as plt
from skimage import io
import os
from os.path import expanduser
HOME = expanduser("~")
SCALED =  'programming/dk43/scaled'
INVERTED = 'programming/dk43/inverted'
ROTATED = 'programming/dk43/rotated'
NORMALIZED = 'programming/dk43/normalized'
MASKED = 'programming/dk43/masked'

In [ ]:
def is_rectangle(c):
    # approximate the contour    
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.001 * peri, True)
    # the contour is 'bad' if it is not a rectangle
    result = len(approx) == 4
    return result

def is_barcode(c):
    x,y,w,h = cv2.boundingRect(c)
    #print ('rectangle size: x=%s y=%s w=%s h=%s' %(x,y,w,h))
    result = (x > 100 and y > 100 and w > 10 and h < 100)
    return result

def norm(img):
    flat = img.flatten()
    hist,bins = np.histogram(flat,256)
    cdf = hist.cumsum() #cumulative distribution function
    cdf = 255 * cdf / cdf[-1] #normalize
    #use linear interpolation of cdf to find new pixel values
    img_norm = np.interp(flat,bins[:-1],cdf)
    img_norm = np.reshape(img_norm, img.shape)
    return img_norm

In [ ]:
def mask_rect(original):
    copied = np.copy(original)
    #original = cv2.normalize(original, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    #img_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    #img_blur = cv2.GaussianBlur(img_gray,(5,5),0)
    #img_bgr = cv2.cvtColor(img_blur,cv2.COLOR_GRAY2BGR)
    """
    img_hsv = cv2.cvtColor(original, cv2.COLOR_BGR2HSV)

    lowHue = 0
    lowSat = 0
    lowVal = 5
    highHue = 55
    highSat = 50
    highVal = 50

    # creation of mask
    colorLow = np.array([lowHue,lowSat,lowVal])
    colorHigh = np.array([highHue,highSat,highVal])
    img_mask = cv2.inRange(img_hsv, colorLow, colorHigh)
    """
    # find  contours
    #contours = cv2.findContours(original, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    edged = cv2.Canny(gray, 50, 100)
    contours = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    
    mx = (0,0,0,0)      # biggest bounding box so far
    mx_area = 0
    for c in contours:
        if is_rectangle(c):
            #print('is rectangle')
            x,y,w,h = cv2.boundingRect(c)
            #print ('rectangle size: x=%s y=%s w=%s h=%s' %(x,y,w,h))
        if is_barcode(c):
            #print('is barcode')
            pass

        if is_rectangle(c) and is_barcode(c):
            print('found barcode and rect')
            copied = cv2.fillPoly(copied, [c], [255,255,0])

    #find largest contour
    #contour_sizes = [(cv2.contourArea(contour), contour) for contour in contours]
    #biggest_contour = max(contour_sizes, key=lambda x: x[0])[1]
    #result = original.copy()
    #if is_barcode(biggest_contour):        
    #    result = cv2.fillPoly(copied, [biggest_contour] , [0,255,0])
    #else:
    #    return result
    return copied

In [ ]:
# normalize the image
dir = os.path.join(HOME,ROTATED)
files = os.listdir(dir)
files = []
for file in sorted(files):
    filename = os.path.join(HOME, ROTATED, file)
    img = cv2.imread(filename)
    img = mask_rect(img)
    output = os.path.join(HOME, MASKED, file)
    cv2.imwrite(output, img) 

In [ ]:
file = 'DK43_slide020_2020_01_23_798515_C0.tif'
file = 'DK43_slide001_2020_01_27_808119_C0.tif'
file = 'DK43_slide052_2020_01_23_801622_C0.tif'

filename = os.path.join(HOME, ROTATED, file)
original = cv2.imread(filename)
img_norm = cv2.normalize(original, None, alpha=1, beta=255, norm_type=cv2.NORM_MINMAX)
#copied = original
copied = mask_rect(img_norm)
#(x > 200 and y > 500 and w > 200 and y < 100)

In [ ]:
# display images
fig, ax = plt.subplots(nrows=3, sharex=True, figsize=(28, 20))
ax[0].set_title('Original')
ax[0].imshow(original)
ax[1].set_title('Normalized')
ax[1].imshow(img_norm)
ax[2].set_title('Filled rect')
ax[2].imshow(copied)